In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [3]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5

from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars


In [13]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat: 
    df[feat] = factorized_values
  else: 
    df[feat + SUFFIX_CAT] = factorized_values

In [16]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')) )
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [31]:
new_feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
len(new_feats)

xgb_params = {
  'max_depth' : 8, 
  'n_estimators' : 50,
  'learning_rate' : 0.25, 
  'seed' : 0  
}
run_model(xgb.XGBRegressor(**xgb_params), new_feats)

[20:45:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:45:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:45:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8007.264102734834, 40.63569565036163)

##Hyperopt

In [37]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), new_feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

#space
xgb_reg_param = {
    'learning_rate':    hp.choice('learning_rate', np.arange(0.3, 0.5, 0.1)),
    'max_depth':        hp.choice('max_depth', np.arange(7, 10, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             0,
}

#run
best = fmin(obj_func, xgb_reg_param, algo=tpe.suggest, max_evals=25)

best


Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.4, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.4, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.4, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.4, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 8, 'n_estimators': 100, 'objectiv

{'colsample_bytree': 0.9,
 'learning_rate': 0,
 'max_depth': 2,
 'subsample': 0.9500000000000001}

In [0]:
!git config --global user.email "maciej.michalczak@gmail.com"
!git config --global user.name "Maciej"

In [39]:
!ls -la

total 112
drwx------ 2 root root  4096 Mar  2 22:58 data
-rw------- 1 root root 42668 Mar  3 23:10 day2_visualisation.ipynb
-rw------- 1 root root 24052 Mar  4 22:28 day3_simple_model.ipynb
-rw------- 1 root root 34351 Mar  5 23:09 day4.ipynb
drwx------ 4 root root  4096 Mar  2 22:57 .git
-rw------- 1 root root  1799 Mar  2 22:57 .gitignore
-rw------- 1 root root  1064 Mar  2 22:57 LICENSE
-rw------- 1 root root    88 Mar  2 22:57 README.md
